In [8]:
from pathlib import Path
from helpers import list_entries_recusive, read_data_to_dict, write_dict_to_data

In [9]:
# copy files to another direcotry
data_dir = Path('data')
version = '1.3.1'

# de_localization = data_dir / version / 'Localization' / 'de'
en_localization = data_dir / version / 'Localization' / 'en'
ja_localization = data_dir / version / 'Localization' / 'ja'

# de_entries = list_entries_recusive(de_localization)
en_entries = list_entries_recusive(en_localization)
ja_entries = list_entries_recusive(ja_localization)

In [18]:

# corpse_entries = [
#     e for e in ja_entries
#     if e.relative_to(ja_localization).as_posix().startswith('Dialog/Corpses')
# ]

# npc_entries = [
#     e for e in ja_entries
#     if e.relative_to(ja_localization).as_posix().startswith('Dialog/NPC')
# ]

# global_entries = [
#     e for e in ja_entries
#     if e.relative_to(ja_localization).as_posix().startswith('Dialogues')
# ]

entries = [
    e for e in ja_entries
    if e.relative_to(ja_localization).as_posix().startswith('Dialogues')
]

# entries = [
#     e for e in ja_entries
#     if "NPC_Bertine_ja" in e.relative_to(ja_localization).as_posix()
# ]

print(len(entries))


21


In [19]:
entries

[PosixPath('data/1.3.1/Localization/ja/Dialogues'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/Global_GreetAndGoodbye_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/Global_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/Global_Busy_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/BanditRuins_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/BanditExpedition_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/Global_PwyllConvo_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/CrimesOpenReactions_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/Tests_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/Sleep_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/WorldDialogueGreatTusk_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/Global_Perceptible_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/PlayerComments_ja.loc'),
 PosixPath('data/1.3.1/Localization/ja/Dialogues/Brutus_ja.loc

In [20]:
from typing import Optional

from llm_utils import get_llm_response
from helpers_llm import get_noun_dictionary_prompt

def trans(context, original_translation: Optional[str] = None):
    system_prompt = "あなたは優秀な翻訳者です。"


    specified_content = "NPCと主人公との会話文"
    # "RPGにおけるアイテム名と説明文"
    # "RPGにおける汎用的な独り言や、会話、あるいは地の文"
    #"RPGにおける、スキルの説明文"

    # rules:
    # * 主人公のものと思われる文章は、ため口で、一人称は「私」です。主人公のテキストは、回答をする側であり、短文であることが多いです。
    # * 過剰にフォーマルな表現は避け、口語的な日本語の表現を使うことが望ましいですが、、「〜してんの？」「〜かよ！」といった口語的にすぎる表現は避けてください。
    # * 恐喝や脅迫のための手紙の場合を除き、フォーマルな表現であることが望ましいです。古い文体と思われる場合は、一般的に分かりやすい範囲で古風な表現を使ってください。
    # * NPCは、一人称が「私」で、初老の男性です。丁寧で深みのある言葉で話します。
    # * NPCは、一人称が「私」で、可愛らしい女性です。丁寧で、親しみやすい女言葉で話します。
    # specified_content = "RPGにおける、手紙の内容"
    prompt = f"""
次のRPGのLocalizationファイルを再翻訳、あるいは校正してください。
内容は、{specified_content}です。
再翻訳・校正時、次のルールに従ってください。

ルール：
* 翻訳は、固有名詞辞書を参考にしてください。
* 主人公のものと思われる文章は、ため口で、一人称は「私」です。主人公のテキストは、回答をする側であり、中性的であり、短文であることが多いです。
* NPCは、一人称が「私」で、狩人の女性です。素朴な言葉で話します。
* 再翻訳文は、原文のニュアンスを保ちつつ、自然な日本語になるように心がけてください。
* 原文のニュアンスを保つためであれば、翻訳文には原文にはない日本語的な表現を使うことで、原文のニュアンスをより明確に伝えるようにしてください。
* 一時翻訳は、翻訳の誤りがある可能性があります。翻訳の参考情報としてのみ使用し、翻訳はあくまで原文を基にしてください。
* 一時翻訳がですます調のときは、だ・である調に変更してください。
* キー {{値}} という形式のフォーマットを絶対に変更しないでください。
前後の変更の例として、各キーは次に示すような形式となります。=>の後の内容は、あなたが出力するべき翻訳文を示しています。
このとき、 => を含む、 => より左側は出力しないでください。出力結果に => という文字列が含まれていたら、処理が間違っているはずです。
Plh_42 {{ What are you doing here? }} => Plh_42 {{ ここで何をしている？ }}
Plh_10 {{ So, your job is to guard nothing. }} => Plh_10 {{ つまり、君の仕事は何も守らないことなんだな。 }}
Plh_12 {{ Can't you say anything besides "right"? }} => Plh_12 {{ 「その通り」以外に何か言えないのか？ }}

### 固有名詞辞書
{get_noun_dictionary_prompt()}

{"### 一時翻訳(参考用)" if original_translation else ""}
{original_translation if original_translation else ""}

### 元の英語
{context}

さぁ、息を吸って、リラックスして、翻訳に取り組みましょう！
"""

    result = get_llm_response(
        model_name="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        params_={
            # "stop": ["\n"]
        }
    )
    return result




target_files = entries

skip_until = None
for idx, entry in enumerate(target_files):
    if entry.is_dir():
        continue
    if skip_until and idx < skip_until:
        print(f"skip {idx}/{len(target_files)}", entry)
        continue

    print(f"{idx}/{len(target_files)}", entry)

    ja_txt = entry.read_text()
    ja_dict = read_data_to_dict(ja_txt)

    en_file = en_localization / entry.relative_to(ja_localization).as_posix().replace('_ja', '_en')
    en_txt = en_file.read_text()
    en_d = read_data_to_dict(en_txt)

    refined_ja_txt = trans(en_txt) #, ja_txt) # 元の翻訳を渡すと、それを参考にして翻訳する
    refined_ja_dict = read_data_to_dict(refined_ja_txt)

    try:
        assert ja_dict.keys() == refined_ja_dict.keys(), f"keys mismatch {entry}"
        entry.write_text(write_dict_to_data(refined_ja_dict))
    except AssertionError as e:
        print(e)
        ja_dict.update(refined_ja_dict)
        entry.write_text(write_dict_to_data(ja_dict))
    except Exception as e:
        raise e


1/21 data/1.3.1/Localization/ja/Dialogues/Global_GreetAndGoodbye_ja.loc
2/21 data/1.3.1/Localization/ja/Dialogues/Global_ja.loc
3/21 data/1.3.1/Localization/ja/Dialogues/Global_Busy_ja.loc
4/21 data/1.3.1/Localization/ja/Dialogues/BanditRuins_ja.loc
5/21 data/1.3.1/Localization/ja/Dialogues/BanditExpedition_ja.loc
6/21 data/1.3.1/Localization/ja/Dialogues/Global_PwyllConvo_ja.loc
7/21 data/1.3.1/Localization/ja/Dialogues/CrimesOpenReactions_ja.loc
8/21 data/1.3.1/Localization/ja/Dialogues/Tests_ja.loc
9/21 data/1.3.1/Localization/ja/Dialogues/Sleep_ja.loc
10/21 data/1.3.1/Localization/ja/Dialogues/WorldDialogueGreatTusk_ja.loc
11/21 data/1.3.1/Localization/ja/Dialogues/Global_Perceptible_ja.loc
12/21 data/1.3.1/Localization/ja/Dialogues/PlayerComments_ja.loc
13/21 data/1.3.1/Localization/ja/Dialogues/Brutus_ja.loc
14/21 data/1.3.1/Localization/ja/Dialogues/CrimesAftermath_ja.loc
15/21 data/1.3.1/Localization/ja/Dialogues/Global_Insults_ja.loc
16/21 data/1.3.1/Localization/ja/Dialogues/